# 4 -----------------------------------------------------------

In [29]:
# Базовые библиотеки
import pathlib
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_absolute_percentage_error

# Базовый ввод данных
DATA_DIR = pathlib.Path("./data")  # Откуда считываем данные
RS = 42
train = pd.read_parquet(DATA_DIR.joinpath("train.parquet"))
CAT = ["feature4"]
TARGETS = ["target0", "target1"]  # список таргетов для модели
FTS = train.filter(like="feature").columns.difference(CAT)  # Выбираем фичи для модели
train["gas"] = 0
train.loc[train.feature4=="gas2", "gas"] = 1
FTS = FTS.union(["gas"])  # список фичей для модели

# разбивка на трейн и тест
X_tr, X_val, y_tr, y_val = train_test_split(train[FTS], train[TARGETS], train_size=0.5, random_state=RS)  

In [2]:
import torch
import torch.nn as nn
import torch.optim as optim
import pandas as pd

In [3]:
# Define the neural network architecture
class RegressionModel(nn.Module):
    def __init__(self):
        super(RegressionModel, self).__init__()
        self.hidden_layers = nn.Sequential(
            nn.Linear(25, 1024),
            nn.ReLU(),
            nn.Linear(1024, 512),
            nn.ReLU(),
            nn.Linear(512, 256),
            nn.ReLU(),
            nn.Linear(256, 2)
        )
        
    def forward(self, x):
        return self.hidden_layers(x)


In [4]:
# Load your data using pandas (assuming you have a DataFrame with 25 features and 2 targets)
# data = pd.read_csv('your_data.csv')

# features = X_tr  
# targets = y_tr  

features = train[FTS]
targets = train[TARGETS]

In [5]:
# Convert the data to PyTorch tensors
features_tensor = torch.tensor(features.values, dtype=torch.float32)
targets_tensor = torch.tensor(targets.values, dtype=torch.float32)

In [6]:
# Move tensors to GPU if available
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
features_tensor = features_tensor.to(device)
targets_tensor = targets_tensor.to(device)

In [7]:
# Create an instance of the regression model
model = RegressionModel()
model = model.to(device)

# Set the model parameters (you can customize these)
learning_rate = 0.001
num_epochs = 200

# Define the loss function and optimizer
criterion = nn.MSELoss()
optimizer = optim.Adam(model.parameters(), lr=learning_rate)

In [8]:
# Training loop
for epoch in range(num_epochs):
    # Forward pass
    outputs = model(features_tensor)
    loss = criterion(outputs, targets_tensor)

    # Backward and optimize
    optimizer.zero_grad()
    loss.backward()
    optimizer.step()

    # Print the loss for every few epochs
    if (epoch + 1) % 10 == 0:
        print(f'Epoch [{epoch+1}/{num_epochs}], Loss: {loss.item():.4f}')

Epoch [10/200], Loss: 89.9595
Epoch [20/200], Loss: 42.8287
Epoch [30/200], Loss: 21.2663
Epoch [40/200], Loss: 15.1361
Epoch [50/200], Loss: 9.8317
Epoch [60/200], Loss: 5.8180
Epoch [70/200], Loss: 3.4715
Epoch [80/200], Loss: 2.1352
Epoch [90/200], Loss: 1.6213
Epoch [100/200], Loss: 1.0615
Epoch [110/200], Loss: 0.8650
Epoch [120/200], Loss: 0.9379
Epoch [130/200], Loss: 0.7513
Epoch [140/200], Loss: 0.6535
Epoch [150/200], Loss: 0.5969
Epoch [160/200], Loss: 0.5498
Epoch [170/200], Loss: 0.5800
Epoch [180/200], Loss: 0.4911
Epoch [190/200], Loss: 0.4729
Epoch [200/200], Loss: 0.4911


In [9]:
# Save the trained model to a file
torch.save(model.state_dict(), 'model.pth')

In [10]:
# Load the saved model
loaded_model = RegressionModel()
loaded_model.load_state_dict(torch.load('model.pth'))
loaded_model = loaded_model.to(device)

In [11]:
test_features = torch.tensor(X_val.values, dtype=torch.float32).to(device)

In [12]:
# Use the loaded model for predictions
# test_features = torch.tensor(test_data.values, dtype=torch.float32).to(device)
predictions = loaded_model(test_features)

In [13]:

predictions = model(test_features)

In [14]:
predictions = predictions.to(torch.device("cpu"))

In [15]:
# predictions.detach().numpy()

In [16]:
print(f"MAPE (val): {mean_absolute_percentage_error(y_val, predictions.detach().numpy()) * 100:.3f} %")

MAPE (val): 2.359 %


In [19]:
loaded_model = RegressionModel()
loaded_model.load_state_dict(torch.load('model.pth'))
loaded_model = loaded_model.to(torch.device("cpu"))

In [21]:
predictions = loaded_model(torch.tensor(X_val.values, dtype=torch.float32))

In [24]:
print(f"MAPE (val): {mean_absolute_percentage_error(y_val, predictions.detach().numpy()) * 100:.3f} %")

MAPE (val): 2.359 %


In [23]:
predictions

tensor([[80.3161, 43.4385],
        [73.8120, 36.7551],
        [86.8475, 52.2644],
        ...,
        [24.1283,  7.2233],
        [21.0981,  1.4661],
        [27.9126,  3.6389]], grad_fn=<AddmmBackward0>)

In [25]:
model = model.to(torch.device("cpu"))
torch.save(model.state_dict(), 'model.pth')